# Introducción

Cuando trabajamos con texto existen multitud de formas de representar la información.

<img src=https://miro.medium.com/max/904/1*DocMTV7nTAomKxcu3m-tyw.jpeg>

# 1. One-Hot Encoding

Antes, introducimos el concepto de **Bag-of-Words**

Quizá la forma más sencilla de representar la información. Permite representar cada texto como un vector. Los pasos son los siguientes:

1. Definir un **vocabulario** (puede extraerse del corpus)
2. Asignamos un entero a cada palabra, de manera que tendremos un vector de longitud igual al número de palabras (cardinalidad) del vocabulario. **Cada posición en el vector representará una palabra del vocabulario**.
3. Para cada documento, asignamos en la posición correspondiente del vector pre-construído a cada palabra que lo compone un valor. Dicho valor puede ser si aparece o no (**Term Presence**) o el número de veces que aparece (**Term Frequency**).

En su aproximación más simple, **one-hot-encoding**, la codificación se realiza a nivel de token. De esta manera, un documento estará definido por N vectores (tantas como tokens contenga), en las que la posición de cada palabra en cada vector tendrá valor igual a 1 (Term Presence).

<img src=https://miro.medium.com/max/1800/1*ArM6Z5jeptCQ082DYn9nDQ.png width=600px>

# 2. Count Vectorizer

Convierte una colleción de documentos en una matriz de documentos-palabras. La codificación se realiza, por tanto, a nivel de documento, en lugar de a nivel de token.

Al ser un modelo de bag-of-words, **no se codifica la información relativa a la posición de los tokens ni su contexto, solo información a si aparecen y su frecuencia**.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
sent_1 = 'me gustan los perros'
sent_2 = 'hay perros y perros'
sent_3 = 'hay muchas razas de perros'

In [ ]:
corpus = [sent_1, sent_2, sent_3]

### Ejemplo básico

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
X = vectorizer.fit_transform(corpus)

In [ ]:

print(vectorizer.get_feature_names_out())



In [ ]:
import pandas as pd
doc_term_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
doc_term_matrix

### Stop words

El parámetro `stop_words` acepta:
- 'english'
- lista de stopwords
- None (default), no filtra stop words

In [ ]:
vectorizer = CountVectorizer(stop_words=['de', 'hay', 'los', 'me'])
X = vectorizer.fit_transform(corpus)

In [ ]:
print(vectorizer.get_feature_names_out())

In [ ]:
doc_term_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
doc_term_matrix

### Número máximo de palabras

El parámetro `max_features` establece el número máximo de features a extraer (vocabulario). Mantendrá solo el top indicado por dicho parámetro.

In [ ]:
vectorizer = CountVectorizer(max_features=4)
X = vectorizer.fit_transform(corpus)

In [ ]:
print(vectorizer.get_feature_names_out())

In [ ]:
doc_term_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
doc_term_matrix

### N-grams como features

El parámetro `ngram_range` (tupla) permite definir los valores de `n` para los ngrams (mínimo y máximo) que serán calculados. Por defecto `ngram_range=(1, 1)` (solo palabras).

In [ ]:
vectorizer = CountVectorizer(ngram_range=(2, 3))  # Jugar con los valores
X = vectorizer.fit_transform(corpus)

In [ ]:
print(vectorizer.get_feature_names_out())

In [ ]:
doc_term_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
doc_term_matrix

### max_df y min_df

Límites superior (`max_df`) e inferior (`min_df`). Pueden definirse como `float` (de 0.0 a 1.0) o como `int`:
- `float`: frecuencia de repetición máxima / mínima
- `int`: número de repeticiones máximo / mínimo

In [ ]:
vectorizer = CountVectorizer(max_df=0.95, min_df=1)  # Jugar con los valores
X = vectorizer.fit_transform(corpus)

In [ ]:
print(vectorizer.get_feature_names_out())

In [ ]:
doc_term_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
doc_term_matrix

### TF-IDF Vectorizer

TF-IDF (Term Frequency - Inverse Document Frequency) es una medida de feature weighting que expresa lo **relevante que es una palabra en un documento**, siendo este documento parte de un corpus.

Tiene en cuenta el número de veces que aparece la palabra (o token) en dicho documento, pero también el total de veces que aparece en todo el corpus.

- **Tokens muy frecuentes a nivel de documento y de corpus** - posibles stop words - obtendrán un valor de **TF-IDF bajo**.
- Tokens que aparecen **solo en ciertos documentos del corpus** tendrán un **IDF mayor** que aquellos que aparecen en mayor número de documentos.

<img src=https://3.bp.blogspot.com/-u928a3xbrsw/UukmRVX_JzI/AAAAAAAAAKE/wIhuNmdQb7E/s1600/td-idf-graphic.png width=700px>

En un sistema de Information Retrieval sencillo, el módulo de ranking de documentos puede construirse considerando el peso de cada documento como la suma de los TF-IDF de cada palabra que lo componen.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [ ]:
transformer = TfidfTransformer()
tf_idf = transformer.fit_transform(X)

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
doc_term_matrix = pd.DataFrame(tf_idf.toarray(), columns=vectorizer.get_feature_names_out())
doc_term_matrix

### TF-IDF Vectorizer (manera directa)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(corpus)

In [ ]:
doc_term_matrix = pd.DataFrame(tf_idf.toarray(), columns=vectorizer.get_feature_names_out())
doc_term_matrix

## Ejemplo: Detección de Spam



#### Lectura de datos

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('./spam.csv', encoding='latin-1')

df = df[['v1', 'v2']]
df.rename(columns={'v1': 'label', 'v2': 'sms'}, inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['label'].value_counts()

#### Preprocesado

In [ ]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

In [ ]:
df.head()

In [ ]:
df['sms'] = df['sms'].str.lower()

In [ ]:
df.head()

#### Train / Test set

In [ ]:
import numpy as np
msk = np.random.rand(len(df)) < 0.75

In [ ]:
df_train = df[msk]
df_test = df[~msk]

In [ ]:
df_train.shape

In [ ]:
df_test.shape

#### Features

In [ ]:
# CountVectorizer simple
cv_simple = CountVectorizer()
X_train_cv_simple = cv_simple.fit_transform(df_train['sms'])
X_test_cv_simple = cv_simple.transform(df_test['sms'])

# CountVectorizer con ngrams, max_features, min_df y max_df
cv_complex = CountVectorizer(ngram_range=(1, 2), max_features=1000, max_df=0.95, min_df=5)
X_train_cv_complex = cv_complex.fit_transform(df_train['sms'])
X_test_cv_complex = cv_complex.transform(df_test['sms'])

# TfIdfVectorizer simple
tfidf_simple = TfidfVectorizer()
X_train_tfidf_simple = tfidf_simple.fit_transform(df_train['sms'])
X_test_tfidf_simple = tfidf_simple.transform(df_test['sms'])

# TfIdfVectorizer complejo
tfidf_complex = TfidfVectorizer(ngram_range=(1, 2), max_features=1000, max_df=0.95, min_df=5)
X_train_tfidf_complex = tfidf_complex.fit_transform(df_train['sms'])
X_test_tfidf_complex = tfidf_complex.transform(df_test['sms'])

#### Modelo de clasificación binaria

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
lr_cv_simple = LogisticRegression()
lr_cv_complex = LogisticRegression()
lr_tfidf_simple = LogisticRegression()
lr_tfidf_complex = LogisticRegression()

In [ ]:
lr_cv_simple.fit(X_train_cv_simple, df_train['label'])  # train
y_pred_cv_simple = lr_cv_simple.predict(X_test_cv_simple)  # test

In [ ]:
lr_cv_complex.fit(X_train_cv_complex, df_train['label'])  # train
y_pred_cv_complex = lr_cv_complex.predict(X_test_cv_complex)  # test

In [ ]:
lr_tfidf_simple.fit(X_train_tfidf_simple, df_train['label'])  # train
y_pred_tfidf_simple = lr_tfidf_simple.predict(X_test_tfidf_simple)  # test

In [ ]:
lr_tfidf_complex.fit(X_train_tfidf_complex, df_train['label'])  # train
y_pred_tfidf_complex = lr_tfidf_complex.predict(X_test_tfidf_complex)  # test

In [ ]:
print('CountVectorizer simple\n')
print(confusion_matrix(df_test['label'], y_pred_cv_simple))
print(classification_report(df_test['label'], y_pred_cv_simple))

In [ ]:
print('CountVectorizer complejo\n')
print(confusion_matrix(df_test['label'], y_pred_cv_complex))
print(classification_report(df_test['label'], y_pred_cv_complex))

In [ ]:
print('TfIdfVectorizer simple\n')
print(confusion_matrix(df_test['label'], y_pred_tfidf_simple))
print(classification_report(df_test['label'], y_pred_tfidf_simple))

In [ ]:
print('TfIdfVectorizer complejo\n')
print(confusion_matrix(df_test['label'], y_pred_tfidf_complex))
print(classification_report(df_test['label'], y_pred_tfidf_complex))

# 3. Word Embeddings

Permiten codificar la información semántica de los tokens en función del contexto (tokens anteriores y posteriores) en el que se encuentren.

Cada palabra estará representada por un vector con dicha información semántica. Operaciones con vectores, y el concepto de distancia, nos permitirá encontrar tokens que semánticamente son parecidos o diferentes.

Lo veremos con más detalle en la próxima sesión.

<img src=https://blog.enzymeadvisinggroup.com/hs-fs/hubfs/Word%20Embeddings%20en%20el%20Natural%20Language%20Processing.png>